In [1]:
import os
import numpy as np # to process the data
import random
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_path = os.path.join(self.directory, img)
                with Image.open(img_path) as img_array:  # read the image
                    gray_array = img_array.convert('L')  # ensure image is grayscale
                    resized_array = gray_array.resize((150, 150))  # resize the image
                    self.images.append([list(resized_array.getdata())[i:i+150] for i in range(0, 22500, 150)])  # keep image in 2D
                    label = img.split('_')[0]  # extract label from filename
                    self.labels.append(label)
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        # Normalize pixel values
        self.images = [[[pixel / 255.0 for pixel in row] for row in image] for image in self.images]
        print(f"Loaded {len(self.images)} images")

# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

# Print the shape of the preprocessed images
print(f"Shape of the preprocessed images: {len(dataset.images)} images, each with {len(dataset.images[0])}x{len(dataset.images[0][0])} pixels")

# Dimensions of the first preprocessed image
print(f"Dimensions of the first preprocessed image: {len(dataset.images[0])}x{len(dataset.images[0][0])} pixels")

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# Custom function to split the dataset
def custom_train_test_split(data, labels, train_size):
    train_count = int(len(data) * train_size)
    return data[:train_count], data[train_count:], labels[:train_count], labels[train_count:]

# train is now 75% of the entire data set
x_train, x_temp, y_train, y_temp = custom_train_test_split(dataset.images, dataset.labels, train_ratio)

# test is now 10% of the initial data set, validation is now 15%
val_count = int(len(x_temp) * (validation_ratio / (test_ratio + validation_ratio)))
x_val, x_test = x_temp[:val_count], x_temp[val_count:]
y_val, y_test = y_temp[:val_count], y_temp[val_count:]

# Print the sizes of the datasets
print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Print the unique classes in the training set
unique_classes = set(y_train)
print(f"Unique classes in the training set: {unique_classes}")


Loaded 10 images
Shape of the preprocessed images: 10 images, each with 150x150 pixels
Dimensions of the first preprocessed image: 150x150 pixels
Train set size: 7, Validation set size: 1, Test set size: 2
Unique classes in the training set: {'hydrangeas', 'orchids', 'lilies', 'tulip', 'hibiscus', 'daisies', 'peonies'}


In [5]:
class Computing:
    def __init__(self, input_layer, output_layer):
        self.input_layer = input_layer
        self.output_layer = output_layer
        self.weights = [[random.random() for _ in range(output_layer)] for _ in range(input_layer)]
        self.biases = [random.random() for _ in range(output_layer)]

    def relu(self, x):
        return max(0, x)

    def relu_derivative(self, x):
        return 1 if x > 0 else 0

    def softmax(self, x):
        e_x = [math.exp(i - max(x)) for i in x]
        return [i / sum(e_x) for i in e_x]

    def cross_entropy(self, y_true, y_pred):
        return -sum([y_true[i]*math.log(y_pred[i]) for i in range(len(y_true))])

class Convolution:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[[random.random() for _ in range(self.filter_size)] for _ in range(self.filter_size)] for _ in range(self.num_filters)]

    def forward(self, input):
        self.last_input = input
        h, w = len(input), len(input[0])
        output = [[[0 for _ in range(self.num_filters)] for _ in range(w - self.filter_size + 1)] for _ in range(h - self.filter_size + 1)]
        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                for f in range(self.num_filters):
                    for i2 in range(self.filter_size):
                        for j2 in range(self.filter_size):
                            output[i][j][f] += input[i+i2][j+j2] * self.filters[f][i2][j2]
        return output

    def backward(self, d_L_d_out, learn_rate):
        d_L_d_input = [[[0]*len(self.last_input[0][0]) for _ in range(len(self.last_input[0]))] for _ in range(len(self.last_input))]
        for i in range(len(d_L_d_out)):
            for j in range(len(d_L_d_out[0])):
                for f in range(len(d_L_d_out[0][0])):
                    for i2 in range(self.filter_size):
                        for j2 in range(self.filter_size):
                            for f2 in range(len(self.filters[0][0][0])):
                                d_L_d_input[i+i2][j+j2][f2] += d_L_d_out[i][j][f] * self.filters[f][i2][j2][f2]
        return d_L_d_input


class MaxPooling:
    def __init__(self, filter_size):
        self.filter_size = filter_size

    def forward(self, input):
        self.last_input = input
        h, w, _ = len(input), len(input[0]), len(input[0][0])
        
        # Calculate the dimensions of the output
        h_out = h // self.filter_size
        w_out = w // self.filter_size
        
        # Initialize the output
        output = [[[0]*_ for _ in range(w_out)] for _ in range(h_out)]
        
        # Perform max pooling
        for i in range(h_out):
            for j in range(w_out):
                for f in range(_):
                    # Flatten the 2D list into a 1D list before applying the max function
                    flat_list = [input[i*self.filter_size+x][j*self.filter_size+y][f] for x in range(self.filter_size) for y in range(self.filter_size)]
                    # Check if flat_list is a list of lists or a list of arrays
                    if isinstance(flat_list[0], list) or isinstance(flat_list[0], np.ndarray):
                        # Flatten the list of lists or list of arrays into a 1D list
                        flat_list = [item for sublist in flat_list for item in sublist]
                    output[i][j][f] = max(flat_list)
        
        return output

    def backward(self, d_L_d_out):
        d_L_d_input = np.zeros(self.last_input.shape)
        for i in range(0, self.last_input.shape[0], self.filter_size):
            for j in range(0, self.last_input.shape[1], self.filter_size):
                h, w, f = self.last_input[i:i+self.filter_size, j:j+self.filter_size].shape
                max_val = np.amax(self.last_input[i:i+self.filter_size, j:j+self.filter_size], axis=(0, 1))
                for i2 in range(h):
                    for j2 in range(w):
                        for f2 in range(f):
                            if self.last_input[i+i2, j+j2, f2] == max_val[f2]:
                                d_L_d_input[i+i2, j+j2, f2] = d_L_d_out[i // self.filter_size, j // self.filter_size, f2]
        return d_L_d_input

class Dense:
    def __init__(self, input_len, output_len):
        self.weights = [[random.random() for _ in range(output_len)] for _ in range(input_len)]
        self.biases = [random.random() for _ in range(output_len)]

    def forward(self, input):
        self.last_input_shape = (len(input), len(input[0]), len(input[0][0]))
        input = [item for sublist in input for item in sublist]
        input = [item for sublist in input for item in sublist]
        self.last_input = input
        input_len, output_len = len(self.weights), len(self.weights[0])
        output = [0]*output_len
        for i in range(input_len):
            for j in range(output_len):
                output[j] += self.weights[i][j] * input[i]
        # Apply the activation function here
        output = [self.relu(o + b) for o, b in zip(output, self.biases)]
        return output

    def softmax(self, x):
        e_x = [math.exp(i - max(x)) for i in x]
        return [i / sum(e_x) for i in e_x]
    
    def backward(self, d_L_d_out, learn_rate):
        d_L_d_input = [0]*len(self.last_input)
        for i, grad in enumerate(d_L_d_out):
            for j in range(len(self.weights)):
                self.weights[j][i] -= learn_rate * d_L_d_out[i] * self.last_input[j]
                d_L_d_input[j] += self.weights[j][i] * d_L_d_out[i]
        return d_L_d_input


class Dropout:
    def __init__(self, drop_probability):
        self.drop_probability = drop_probability

    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        output = [0]*len(input)
        self.dropped = []
        for i in range(len(input)):
            if random.random() < self.drop_probability:
                output[i] = 0
                self.dropped.append(True)
            else:
                output[i] = input[i]
                self.dropped.append(False)
        return output

    def backward(self, d_L_d_out, learn_rate):
        return [d_L_d_out[i] if not self.dropped[i] else 0 for i in range(len(d_L_d_out))]


class Network:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def compute_loss(self, y_true, y_pred):
        # Cross-entropy loss
        return -sum([y_true[i]*math.log(y_pred[i]) for i in range(len(y_true))])

    def loss_derivative(self, y_true, y_pred):
        return [-y_true[i]/y_pred[i] for i in range(len(y_true))]

    def compute_accuracy(self, y_true, y_pred):
        # Convert predictions to one-hot vectors
        y_pred = [1 if p == max(y_pred) else 0 for p in y_pred]

        # Compute accuracy
        correct_predictions = sum([1 if y_true[i] == y_pred[i] else 0 for i in range(len(y_true))])
        accuracy = correct_predictions / len(y_true)
        return accuracy

    def backward(self, X, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(X, grad)

    def update(self, learning_rate):
        for layer in self.layers:
            if isinstance(layer, Dense):
                for i in range(len(layer.weights)):
                    for j in range(len(layer.weights[0])):
                        layer.weights[i][j] -= learning_rate * layer.weights[i][j]
                for i in range(len(layer.biases)):
                    layer.biases[i] -= learning_rate * layer.biases[i]

    def train(self, X, y, epochs, learning_rate):
        for e in range(epochs):
            out = self.forward(X)
            grad = self.loss_derivative(y, out)
            self.backward(X, grad)
            self.update(learning_rate)

In [ ]:
def create_model():
    # Initialize the network
    network = Network()

    # Add layers to the network
    # Adjust the parameters as needed
    network.add_layer(Convolution(num_filters=32, filter_size=3))
    network.add_layer(MaxPooling(filter_size=2))
    network.add_layer(Convolution(num_filters=64, filter_size=3))
    network.add_layer(MaxPooling(filter_size=2))
    network.add_layer(Dense(input_len=64*37*37, output_len=128))  # Adjust input_len based on the output of the last MaxPooling layer
    network.add_layer(Dropout(drop_probability=0.5))
    network.add_layer(Dense(input_len=128, output_len=10))  # 10 output classes for 10 types of flowers

    return network

In [ ]:
def train_model(model, x_train, y_train, epochs, learning_rate, threshold, early_stoppings):
    # Initialize variables for early stopping
    best_loss = float('inf')
    rounds_without_improvement = 0

    # Loop over the epochs
    for epoch in range(epochs):
        # Forward pass and compute loss
        output = model.forward(x_train)
        loss = model.compute_loss(y_train, output)

        # Compute accuracy
        accuracy = model.compute_accuracy(y_train, output)

        # Print loss and accuracy for this epoch
        print(f"Epoch {epoch+1}/{epochs} -- Loss: {loss} - Accuracy: {accuracy}")

        # Check for early stopping
        if loss < best_loss - threshold:
            best_loss = loss
            rounds_without_improvement = 0
        else:
            rounds_without_improvement += 1
            if rounds_without_improvement == early_stopping:
                print("Early stopping due to no improvement after", early_stopping, "rounds.")
                break

        # Backward pass and update weights
        grad = model.loss_derivative(y_train, output)
        model.backward(x_train, grad)
        model.update(learning_rate)


In [ ]:
# Create the model
model = create_model()

# Define your training parameters
epochs = 10  # Number of epochs to train for
learning_rate = 1  # Learning rate
threshold = 0.01  # Threshold for early stopping
early_stopping = 10  # Number of rounds without improvement before early stopping

# Train the model
train_model(model, x_train, y_train, epochs, learning_rate, threshold, early_stopping)
